In [14]:
import requests
from requests import Response
from pprint import pprint
from pydantic import BaseModel, RootModel, Field,field_validator, field_serializer
from datetime import datetime
import numpy as np
import pandas as pd

youbike_url = 'https://data.ntpc.gov.tw/api/datasets/c51d5111-c300-44c9-b4f1-4b28b9929ca2/json?size=1000'
try:
    respons:Response = requests.request('GET',youbike_url)
    respons.raise_for_status()
except Exception:
    print("下載錯誤")
else:
    content = respons.text

class Factory(BaseModel):
    名稱: str = Field(alias='organizer')
    註冊號碼: str = Field(alias='no')
    地址: str = Field(alias='address')
    統編: str = Field(alias='tax_id_number')
    緯度: float|None = Field(alias='wgs84ax') # Field 的資料類型可以設為 xxxx:float | None
    經度: float|None = Field(alias='wgs84ay') # Field 的資料類型可以設為 xxxx:float | None
    日期: datetime = Field(alias='date')

    @field_validator('緯度', '經度', mode='before') # 先自訂驗證
    @classmethod
    def validate_coordinates(cls, value):
        if value == None:
            return None
        else:
            return round(float(value), ndigits=5)

    @field_serializer('日期')
    def date_serial(self, date: datetime) -> str:
        return date.strftime(f'中華民國{date.year-1911}年%m月%d日 %H:%M:%S')

class Companys(RootModel):
    root: list[Factory]

companys: Companys = Companys.model_validate_json(content)
companys_list = companys.model_dump()
pprint(companys_list)
df1 = pd.DataFrame(companys_list)
df1

#to_csv()
df1.to_csv('factory.csv',index=False, encoding='utf-8')

#to_excel()
df1.to_excel('factory.xlsx',sheet_name='工廠名單')

                 名稱      註冊號碼                   地址        統編         緯度  \
0             鴻輝食品廠  99611344   新北市新莊區新莊路四九三巷四弄一一號  35567635  121.44862   
1       廣達香食品股份有限公司  99604192     新北市新莊區化成路三八二巷一八號  35549807  121.46722   
2             順發糧食行  99613990     新北市板橋區縣民大道一段二八一號  33305209  121.46050   
3      台灣塩野香料股份有限公司  99600477        新北市新莊區化成路二一九號   3561504  121.46569   
4      圓金釀造食品股份有限公司  99606467     新北市新莊區新樹路二六八巷二二號  35427115  121.43106   
..              ...       ...                  ...       ...        ...   
995    宇笛生物科技股份有限公司  65007250   新北市中和區立德街148巷52號6樓  12953312  121.48631   
996        芊語實業有限公司  65007281      新北市板橋區信義路165號2樓  66639825  121.45882   
997  海邊走走食品股份有限公司二廠  65007312  新北市淡水區中正東路1段3巷23號1樓  54710204  121.45188   
998        草系生活有限公司  65007319   新北市土城區中央路3段87號7樓之1  54151784  121.43784   
999        明朝製麵有限公司  65007320       新北市樹林區忠信街15號3樓  50845258  121.40782   

           經度                      日期  
0    25.03270  中華民國59年08月23日 03:38:51  
1    25.04990  中華民國